# Convert flux to counts

### This will convert the FITS images from units of flux to counts. Depending on the input of the conversion function will depend on what the function does. It can take in either nothing (which means do all files), a .fits ending (which means do a single file) or a .txt ending (which means do multiple files)

#### To convert the data from flux to counts, it uses the conversion equation counts = flux * (exptime/fluxconv) where exptime and fluxconv factor is found in the FITS header

In [1]:
# Import modules
import re
import shutil
from astropy.io import fits
import os
import fnmatch

In [2]:
# Input - what to be converted
input = ''

In [3]:
# Determine what type of file the input it
if input.endswith('.fits'):
    print "It is a single image"
elif input.endswith('.txt'):
    print "It is a text file containing multiple images"
elif input == "":
    print "No input - converting all files"

No input - converting all files


In [4]:
def convert_single_image(input):
    
    # Convert a single file - '.fits' ending
    
    new_name = re.sub('.fits', '_dn.fits', input)
    
    # Only if the dn file doesn't exist does it do this
    if os.path.isfile(new_name) == False:
        shutil.copy(input, new_name)
        hdulist = fits.open(new_name, mode='update')
        prihdr = hdulist[0].header
        scidata = hdulist[0].data
        exptime = prihdr['exptime']
        fluxconv = prihdr['fluxconv']
        scidata *= exptime/fluxconv

In [6]:
def convert_multiple_images(input):
    # Convert multiple file - '.txt' ending
    f = open(input, 'r')
    files = f.read().splitlines()
    
    for image in files:
        convert_single_image(image)

    f.close()

In [5]:
# Convert all files - no input
def convert_all_files(input):
    
    home = '/home/ac833/Data/LMC/'
    os.chdir(home)
    
    # Finds all directories - these are all the r... folders that need to be sorted
    directories = os.listdir(home)

    for directory in directories:
        for root,dirs,files in os.walk(directory):
            for filename in files:
                if fnmatch.fnmatch(filename,'*_cbcd.fits'): # find all the BCD images
                    convert_single_image(os.path.join(root,filename))


In [7]:
# Convert corresponding files to counts
if input.endswith('.fits'):
    convert_single_image(input)
elif input.endswith('.txt'):
    convert_multiple_images(input)
elif input == "":
    convert_all_files(input)

## Finished - the input should now be converted to counts